In [1]:
# 필요한 패키지와 라이브러리를 가져옴
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
import numpy as np
# conda install -c conda-forge fbprophet 
from prophet import Prophet

mpl.rc('font',family='Malgun Gothic')   # 맑은 고딕으로 그래프한글설정

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False

In [2]:
## file 불러옴
data = pd.read_csv('./data/Test.csv', encoding = 'cp949')
data.head()

,date,car_num,daily_distance,cum_distance,information_date,car_speed,RPM,brake_signal,car_location_GPS_X,car_location_GPS_Y,RPMS,ACC,FS,RPMSR,ACCR,SRA
0,20221201,2612,44,3093066,2.212190e+13,0,577,0,129.158634,35.236121,-29.5,0.0,0,0,0,0
1,20221201,2612,44,3093066,2.212190e+13,0,555,0,129.158634,35.236121,-1.5,0.0,0,0,0,0
2,20221201,2612,44,3093066,2.212190e+13,0,558,0,129.158634,35.236121,4.5,0.0,0,0,0,0
3,20221201,2612,44,3093066,2.212190e+13,0,546,0,129.158634,35.236121,-6.5,0.0,0,0,0,0
4,20221201,2612,44,3093066,2.212190e+13,0,561,0,129.158634,35.236121,2.0,0.0,0,0,0,0


In [3]:
type(data)

pandas.core.frame.DataFrame

In [4]:
#필요한 열 추출
data = data[['date','car_num','daily_distance']]
data

,date,car_num,daily_distance
0,20221201,2612,44
1,20221201,2612,44
2,20221201,2612,44
3,20221201,2612,44
4,20221201,2612,44
...,...,...,...
168083,20221214,9357,88
168084,20221214,9357,88
168085,20221214,9357,88
168086,20221214,9357,88


In [5]:
# 날짜별, 차량별로 일일주행거리 최대값만 추출
data1 = data.loc[data.groupby(['date','car_num'])['daily_distance'].idxmax()]
data1

,date,car_num,daily_distance
1288,20221201,2612,88
33571,20221201,3897,88
48697,20221201,4699,44
65235,20221201,4985,44
81829,20221201,5221,135
...,...,...,...
113548,20221214,5634,220
131547,20221214,8993,44
149314,20221214,9007,176
168074,20221214,9357,88


In [8]:
data1['oil_money'] = data1['daily_distance'] * 800
data1

,date,car_num,daily_distance,oil_money
1288,20221201,2612,88,70400
33571,20221201,3897,88,70400
48697,20221201,4699,44,35200
65235,20221201,4985,44,35200
81829,20221201,5221,135,108000
...,...,...,...,...
113548,20221214,5634,220,176000
131547,20221214,8993,44,35200
149314,20221214,9007,176,140800
168074,20221214,9357,88,70400


In [9]:
# 컬럼 리스트 생성
date = data1.iloc[:,0].to_list()
car_num = data1.iloc[:,1].to_list()
daily_distance = data1.iloc[:,2].to_list()
oil_money = data1.iloc[:,3].to_list()

In [10]:
list_sqlData = []
# sql문 생성
for i in range(len(data1)) :
    a = date[i], car_num[i], daily_distance[i], oil_money[i]
    list_sqlData.append(a)
list_sqlData

[(20221201, 2612, 88, 70400),
 (20221201, 3897, 88, 70400),
 (20221201, 4699, 44, 35200),
 (20221201, 4985, 44, 35200),
 (20221201, 5221, 135, 108000),
 (20221201, 5634, 44, 35200),
 (20221201, 8993, 132, 105600),
 (20221201, 9007, 176, 140800),
 (20221201, 9357, 44, 35200),
 (20221201, 9582, 44, 35200),
 (20221202, 2612, 133, 106400),
 (20221202, 3897, 132, 105600),
 (20221202, 4699, 88, 70400),
 (20221202, 4985, 88, 70400),
 (20221202, 5221, 44, 35200),
 (20221202, 5634, 88, 70400),
 (20221202, 8993, 176, 140800),
 (20221202, 9007, 220, 176000),
 (20221202, 9357, 88, 70400),
 (20221202, 9582, 88, 70400),
 (20221203, 2612, 177, 141600),
 (20221203, 3897, 176, 140800),
 (20221203, 4699, 132, 105600),
 (20221203, 4985, 132, 105600),
 (20221203, 5221, 88, 70400),
 (20221203, 5634, 132, 105600),
 (20221203, 8993, 220, 176000),
 (20221203, 9007, 44, 35200),
 (20221203, 9357, 132, 105600),
 (20221203, 9582, 132, 105600),
 (20221204, 2612, 221, 176800),
 (20221204, 3897, 220, 176000),
 (2022

In [13]:
## list를 MySQL DB에 넣음
import pymysql
conn = pymysql.connect(host='database-1.cd1ds11df3el.ap-northeast-2.rds.amazonaws.com',
                                     database='p1',
                                     user='admin',
                                     password='tigertiger')
try:
    with conn.cursor() as cursor:
        sql = 'INSERT INTO Oil (date, car_num, daily_distance, oil_money) VALUES (%s, %s, %s, %s)'
        cursor.executemany(sql, list_sqlData)
    conn.commit()
finally:
    conn.close()